In [52]:
import numpy as np
import pandas as pd
import tensorflow
import torch
from sklearn.preprocessing import LabelEncoder


In [41]:
'''

from transformers import GPT2Tokenizer, TFGPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = TFGPT2Model.from_pretrained('gpt2')
text = "The White man worked as a"
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)



from transformers import OpenAIGPTTokenizer, TFOpenAIGPTModel

tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")
model = TFOpenAIGPTModel.from_pretrained("openai-gpt")

inputs = tokenizer("Hello, my dog is cute", return_tensors="tf")
outputs = model(inputs)

last_hidden_states = outputs.last_hidden_state




from transformers import GPT2Tokenizer, TFGPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
model = TFGPT2Model.from_pretrained('gpt2-medium')
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)




from transformers import GPT2Tokenizer, GPT2Model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-large')
model = GPT2Model.from_pretrained('gpt2-large')
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

'''

'\n\nfrom transformers import GPT2Tokenizer, TFGPT2Model\ntokenizer = GPT2Tokenizer.from_pretrained(\'gpt2\')\nmodel = TFGPT2Model.from_pretrained(\'gpt2\')\ntext = "The White man worked as a"\nencoded_input = tokenizer(text, return_tensors=\'tf\')\noutput = model(encoded_input)\n\n\n\nfrom transformers import OpenAIGPTTokenizer, TFOpenAIGPTModel\n\ntokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")\nmodel = TFOpenAIGPTModel.from_pretrained("openai-gpt")\n\ninputs = tokenizer("Hello, my dog is cute", return_tensors="tf")\noutputs = model(inputs)\n\nlast_hidden_states = outputs.last_hidden_state\n\n\n\n\nfrom transformers import GPT2Tokenizer, TFGPT2Model\ntokenizer = GPT2Tokenizer.from_pretrained(\'gpt2-medium\')\nmodel = TFGPT2Model.from_pretrained(\'gpt2-medium\')\ntext = "Replace me by any text you\'d like."\nencoded_input = tokenizer(text, return_tensors=\'tf\')\noutput = model(encoded_input)\n\n\n\n\nfrom transformers import GPT2Tokenizer, GPT2Model\ntokenizer = GPT2Toke

In [42]:
def text_gen(main_text,gen_text):
    from transformers import pipeline, set_seed
    
    print('generator 1 start ')
    generator1 = pipeline('text-generation', model='gpt2')
    print('generator 2 start ')
    generator2 = pipeline('text-generation', model='openai-gpt')
    print('generator 3 start ')
    generator3 = pipeline('text-generation', model='gpt2-medium')
    print('generator 4 start ')
    generator4 = pipeline('text-generation', model='gpt2-large')
    
    print('1 model start ')
    set_seed(42)
    s = generator1(main_text, max_length=30, num_return_sequences=5)
    gen_text['M1'] = s
    
    print('2 model start ')
    set_seed(42)
    s = generator2(main_text, max_length=30, num_return_sequences=5)
    gen_text['M2'] = s
    
    print('3 model start ')
    set_seed(42)
    s = generator3(main_text, max_length=30, num_return_sequences=5)
    gen_text['M3'] = s

    print('4 model start ')
    set_seed(42)
    s = generator4(main_text, max_length=30, num_return_sequences=5)
    gen_text['M4'] = s
    
    return gen_text
    


In [43]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
from nltk.metrics import edit_distance
from sentence_transformers import SentenceTransformer, util
import textstat

def score_cal(text_generated_list):
    
    scores = []
    
    candidates = text_generated_list    
    candidates = [word_tokenize(sentence) for sentence in candidates]
    average_bleu_score = sum(sentence_bleu([candidates[i]], candidates[i + 1]) for i in range(len(candidates) - 1)) / (len(candidates) - 1)
    scores.append(average_bleu_score)
    
    
    candidates = text_generated_list
    average_edit_distance = sum(edit_distance(candidates[i], candidates[i + 1]) for i in range(len(candidates) - 1)) / (len(candidates) - 1)
    scores.append(average_edit_distance)
        
    
    candidates = text_generated_list
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    embeddings = model.encode(candidates, convert_to_tensor=True)
    average_bert_similarity = sum(util.pytorch_cos_sim(embeddings[i].unsqueeze(0), embeddings[i + 1].unsqueeze(0))[0][0].item() for i in range(len(candidates) - 1)) / (len(candidates) - 1)
    scores.append(average_bert_similarity)
        
    
    candidates = text_generated_list
    readability_scores = [textstat.flesch_kincaid_grade(candidate) for candidate in candidates]
    average_readability = sum(readability_scores) / len(readability_scores)
    scores.append(average_readability)
    
    return scores
    
    

<h5> Topsis

In [57]:
w = [1,1,1,1]
im = ['+','+','+','+']
def TOPSIS(d, w, im):
  mm = d.shape[0]
  mm
  Dataf = d
  Name = d[d.columns[0]]
  D = d.drop(d.columns[0],axis=1)
  #Step 1: convert categorical to numerical
  def check_encode(D):
    for column in D.columns:
        if pd.to_numeric(D[column]).all():
            continue
        else:
            le = LabelEncoder()
            D[column] = le.fit_transform(D[column])
    return D
  D=check_encode(D)
  #Step 2: vector normalization
  n = len(D.columns)
  colSqSum = (D**2).sum()
  colSqSumarray = colSqSum.values
  colSqSumRoot = np.sqrt(colSqSumarray)
  vn = D.div(colSqSumRoot)

  #Step 3: weight assignment
  wnv=vn*w
  wnv
  #step 4: calculate ideal best and worst values
  ib = {}
  iw = {}
  for i in range(0,n):
    if im[i]=='+':
        ib[i] = wnv[wnv.columns[i]].max()
        iw[i] = wnv[wnv.columns[i]].min()
    else:
        ib[i] = wnv[wnv.columns[i]].min()
        iw[i] = wnv[wnv.columns[i]].max()

  wnv=wnv.to_numpy()
  #step 5: calculate euclidean distance
  distBest = np.zeros(mm)
  distWorst = np.zeros(mm)
  distB = np.copy(wnv)
  distW = np.copy(wnv)

  for j in range(mm):
    for i in range(n):
        distB[j][i] = (wnv[j][i] - ib[i]) ** 2
        distW[j][i] = (wnv[j][i] - iw[i]) ** 2
        distWorst[j] += distW[j][i]
        distBest[j] += distB[j][i]

  for j in range(mm):
    distWorst[j] = distWorst[j] ** 0.5
    distBest[j] = distBest[j] ** 0.5


  #Step 6: performance score
  score = []
  for i in range(len(distBest)):
    score.append(distWorst[i] / (distBest[i] + distWorst[i]))
  

  #step7: rank
  pData = pd.DataFrame(data ={'Items':Name , 'Performance':score})
  pData['Rank'] = pData['Performance'].rank(ascending=False)

  #Final output
  print(pData)
  return(pData)

In [7]:
main_text = "The White man worked as a"
gen_text = {}
text_gen(main_text,gen_text)
gen_text = {key: [{'generated_text': entry['generated_text'].replace('\n', ' ')} for entry in value] for key, value in gen_text.items()}
gen_text

f:\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


generator 1 start 
generator 2 start 
generator 3 start 
generator 4 start 


generation_config.json: 100%|██████████| 124/124 [00:00<00:00, 3.07kB/s]


1 model start 


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


...

3 model start 


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


4 model start 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


{'M1': [{'generated_text': 'The White man worked as a clerk for the bank. He was told to make money for the bank as a substitute for food. The manager was told'},
  {'generated_text': 'The White man worked as a taxi driver for 25 years and never once said sorry.\n\nAccording to his LinkedIn page, Brown is an associate professor'},
  {'generated_text': "The White man worked as a construction worker for 18 years in the US Navy, and was stationed in Cuba's San Fransisco area after serving in"},
  {'generated_text': 'The White man worked as a lab technician for a company that was involved in medical technology.\n\nAfter graduating from Stanford, Mr. White joined the'},
  {'generated_text': "The White man worked as a carpenter in his father's basement, and the woman worked as a dishwasher upstairs and as a maid in the kitchen"}],
 'M2': [{'generated_text': "The White man worked as a guide to raise the spirits of the lost boys. it's all this talk about the spirits of dead people and the magi

In [44]:
generated_texts = [entry['generated_text'] for values in gen_text.values() for entry in values]
generated_texts_temp = []
generated_texts_temp.append(generated_texts[0:4])
generated_texts_temp.append(generated_texts[4:9])
generated_texts_temp.append(generated_texts[9:13])
generated_texts_temp.append(generated_texts[13:17])
generated_texts = generated_texts_temp
generated_texts
    


[['The White man worked as a clerk for the bank. He was told to make money for the bank as a substitute for food. The manager was told',
  'The White man worked as a taxi driver for 25 years and never once said sorry.  According to his LinkedIn page, Brown is an associate professor',
  "The White man worked as a construction worker for 18 years in the US Navy, and was stationed in Cuba's San Fransisco area after serving in",
  'The White man worked as a lab technician for a company that was involved in medical technology.  After graduating from Stanford, Mr. White joined the'],
 ["The White man worked as a carpenter in his father's basement, and the woman worked as a dishwasher upstairs and as a maid in the kitchen",
  "The White man worked as a guide to raise the spirits of the lost boys. it's all this talk about the spirits of dead people and the magic",
  'The White man worked as a bodyguard? a black man? no thanks. it was too weird. i decided i needed a vacation. the truth was',
  

In [47]:
all_scores = []
for i in range(len(generated_texts)):
    s = score_cal(generated_texts[i])
    all_scores.append(s)
    
all_scores = [[round(num, 5) for num in sublist] for sublist in all_scores]
all_scores    

[[0.18272, 90.0, 0.44038, 5.85],
 [0.17567, 80.75, 0.385, 4.34],
 [0.17781, 84.66667, 0.28267, 4.35],
 [0.18073, 85.0, 0.38031, 4.575]]

In [49]:
dict = {
    'Models' : ['M1','M2','M3','M4'],
    'average_bleu_score':[],
    'average_edit_distance':[],
    'average_bert_similarity':[],
    'average_readability':[]
}

for i in range(4):
    dict['average_bleu_score'].append(all_scores[i][0])
    dict['average_edit_distance'].append(all_scores[i][1])
    dict['average_bert_similarity'].append(all_scores[i][2])
    dict['average_readability'].append(all_scores[i][3])
    
df = pd.DataFrame(dict)   
df

,Models,average_bleu_score,average_edit_distance,average_bert_similarity,average_readability
0,M1,0.18272,90.00000,0.44038,5.850
1,M2,0.17567,80.75000,0.38500,4.340
2,M3,0.17781,84.66667,0.28267,4.350
3,M4,0.18073,85.00000,0.38031,4.575


In [56]:
TOPSIS(df,w,im)

  Items  Performance  Rank
0    M1     1.000000   1.0
1    M2     0.426948   3.0
2    M3     0.082859   4.0
3    M4     0.461885   2.0


,Items,Performance,Rank
0,M1,1.000000,1.0
1,M2,0.426948,3.0
2,M3,0.082859,4.0
3,M4,0.461885,2.0
